# Hello GeoPandas, Alaaf Aachen! - Data analysis


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Reading/ Parsing all necessary files
Let us load a dataset containing the statistical neighbourhoods of Aachen. It is located in the **/data/aachen** directory from where this notebook is located. GeoPandas supports a wide range of GIS data formats including Shape file (usually ending on .shp).

In [ ]:
bezirke = gpd.read_file("./data/aachen/StatistischeBezirkeAachen.shp")

GeoPandas now has loaded the data from the shapefile with its geometric information and the data on the attributes of the shape into a table. We can inspect the table using the **.plot()** function offered by the GeoPandas DataFrame.

In [ ]:
bezirke.head()

In [ ]:
bezirke.plot()

So far, the data contained in the data set is not very interesting: just the shapes of the borders, an identificator  ("STATBEZ") to identify the area and the name stored in the column ("ST_NAME").

In [ ]:
statistics = pd.read_csv("./data/aachen/einwohnerstatistik-31.12.2020.csv")
statistics.head()
statistics.plot()

## Investigating the information for commonalities
This is a plain CSV Table that has no geographic information attached to it. However, the two DataFrames have a feature in common: the **id** of the area. The column names differ, they are the same entity though. Let's order the tables by the respective id and see whether they match.


In [ ]:
#Resorting the statistic DataFrame
sorted_stats = statistics.sort_values(by='StatBezName')
sorted_stats.head()

In [ ]:
#Resorting the bezirke DataFrame
sorted_bezirke = bezirke.sort_values(by='ST_NAME')
sorted_bezirke.head()

With this information we can add the information from the extended statistics DataFrame we have assigned in `statistics` to the GeoDataFrame. Here, every column of the `statistics` data set is added to the table. With tabular data, this concept is referred to as **merging** or **joining** data. 

See an explanation on merging GeoDataFrames here: https://geopandas.org/docs/user_guide/mergingdata.html 

Here, we are joining the `statistics` data frame to the `bezirke` data from the left

```
bezirke <-- statistics
```

## Renaming columns 
First however, we have to rename the column name `Bez` from the `statistics` DataFrame to match the respective name `STATBEZ` in our `bezirke` DataFrame

In [ ]:
statistics = statistics.rename(columns = {'Bez' : 'STATBEZ'})
statistics.head(20)



## Perparing information

In [ ]:
#pd.to_numeric(statistics["STATBEZ"])
import numpy as np
statistics = statistics.replace(r'^\s*$', np.nan, regex=True)
index = statistics[statistics['STATBEZ'] == np.nan].index
#statistics.drop(34, inplace=True)
statistics.head(20)

Next we have an example for using NumPy dtype. Here we define the column content for __STATBEZ__ as type integer64. Integer, as we want them to be hole numbers not floating point numbers and 64 defines the size of date, here 64 bytes.

In [ ]:
statistics["STATBEZ"] = statistics["STATBEZ"].fillna(0)
statistics["STATBEZ"] = statistics["STATBEZ"].astype('int64')

statistics["STATBEZ"].tail()

## Merging information into one DataFrame

In [ ]:
# inital attempt will result in an error. This is expected, see below for explanation and solution
bezirke_stats  = bezirke.merge(statistics, on='STATBEZ', how='left')
# bezirke_stats = bezirke.set_index('STATBEZ').join(statistics.set_index('STATBEZ'))
bezirke_stats.tail(3)
bezirke_stats = gpd.GeoDataFrame(bezirke_stats)
type(bezirke_stats)
bezirke_stats.head(5)

In [ ]:
# set to min and max of data
vmin, vmax = 0, 133623

# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(14,6))
# add a title and annotation
ax.set_title('Total population of Aachen', fontdict={'fontsize': '25', 'fontweight' : '3'})

ax = bezirke_stats.plot(column='Pers', cmap = 'YlGnBu', ax=ax,   
                                legend =  
                                  True)

In [ ]:
print(bezirke_stats[1:5]["geometry"].area)

In [ ]:
bezirke_stats['popdens'] = bezirke_stats["Pers"] / bezirke_stats["geometry"].area*1000

In [ ]:
bezirke_stats.head()

In [ ]:
# set to min and max of data
vmin, vmax = 0, 133623

# create figure and axes for Matplotlib
fig, ax = plt.subplots(1, figsize=(14,6))
# add a title and annotation
ax.set_title('Population Density of Aachen in 1000 inhabitants per km^2', fontdict={'fontsize': '25', 'fontweight' : '3'})

ax = bezirke_stats.plot(column='popdens', cmap = 'YlGnBu', ax=ax,   
                                legend =  
                                  True)